In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/kevinshuey/massive_dataset/quant_intern/future_price20230628.txt', sep='\t', header=0)
print(df.columns)

Index(['TDATE', 'TTIME', 'UPDATEMILLISEC', 'CONTRACTID', 'ExchangeInstID',
       'CONTRACTNAME', 'LASTPX', 'HIGHPX', 'LOWPX', 'CQ', 'CM', 'OC', 'TQ',
       'TM', 'LASTQTY', 'INITOPENINTS', 'OPENINTS', 'INTSCHG', 'RISELIMIT',
       'FALLLIMIT', 'PRESETTLE', 'PRECLOSE', 'S1', 'B1', 'SV1', 'BV1',
       'OPENPX', 'CLOSEPX', 'SETTLEMENTPX', 'LIFELOW', 'LIFEHIGH', 'AVGPX',
       'BIDIMPLYQTY', 'ASKIMPLYQTY', 'BSRATIO', 'SIDE', 'MFLXID', 'MFLXNAME',
       'S5', 'S4', 'S3', 'S2', 'B2', 'B3', 'B4', 'B5', 'SV5', 'SV4', 'SV3',
       'SV2', 'BV2', 'BV3', 'BV4', 'BV5', 'PREDELTA', 'CURRDELTA', 'LOCALTM',
       'MARKET', 'CHG', 'CHGPCT', 'VARIETIES', 'SETTLEGROUPID', 'SETTLEID',
       'UNIX', 'CLEARINGDAY', 'MFLAG', 'SOURCE', 'CONTRACTCODE', 'Offset',
       'TFLAG'],
      dtype='object')


In [3]:
df_u = df[['TDATE', 'TTIME', 'UPDATEMILLISEC', 
         'LASTPX', 'B1', 'S1', 'AVGPX', 'TQ', 'OPENINTS']]

for i in ['LASTPX', 'B1', 'S1', 'AVGPX', 'TQ', 'OPENINTS']:
    missing = df_u[i].isnull().sum()
    print(f"Missing {i}: {missing}")

Missing LASTPX: 432
Missing B1: 714
Missing S1: 2303
Missing AVGPX: 5344567
Missing TQ: 0
Missing OPENINTS: 0


In [4]:
df_u = df_u.dropna(subset=['LASTPX', 'S1', 'B1', 'AVGPX'])

In [5]:
df_pre = df_u.copy().shift(1)
df_pre["dP"] = df_u["LASTPX"] - df_pre["LASTPX"]
df_pre["dOI"] = df_u["OPENINTS"] - df_pre["OPENINTS"]
df_pre["dVOL"] = df_u["TQ"] - df_pre["TQ"]
df_pre.head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,dP,dOI,dVOL
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230628.0,85900.0,4.0,13125.0,13125.0,13135.0,13125.0,4537.0,78428.0,0.0,-3.0,14.0
2,20230628.0,90000.0,484.0,13125.0,13120.0,13125.0,13120.0,4551.0,78425.0,5.0,5.0,32.0
3,20230628.0,90000.0,975.0,13130.0,13125.0,13135.0,13125.0,4583.0,78430.0,-5.0,0.0,9.0
4,20230628.0,90001.0,476.0,13125.0,13125.0,13130.0,13125.0,4592.0,78430.0,0.0,5.0,9.0


In [6]:
con_buy = (df_u['LASTPX'] >= df_pre['B1']) | (df_u['LASTPX'] >= df_u['B1'])
con_sell = (df_u['LASTPX'] <= df_pre['S1']) |(df_u['LASTPX'] <= df_u['S1'])
dir_val = np.where(con_buy, 'BUY', np.where(con_sell, 'SELL', 'Unclassified'))

df_pre['direction'] = pd.Series(dir_val, index=df_u.index)
df_pre.dropna().head()

,TDATE,TTIME,UPDATEMILLISEC,LASTPX,B1,S1,AVGPX,TQ,OPENINTS,dP,dOI,dVOL,direction
1,20230628.0,85900.0,4.0,13125.0,13125.0,13135.0,13125.0,4537.0,78428.0,0.0,-3.0,14.0,BUY
2,20230628.0,90000.0,484.0,13125.0,13120.0,13125.0,13120.0,4551.0,78425.0,5.0,5.0,32.0,BUY
3,20230628.0,90000.0,975.0,13130.0,13125.0,13135.0,13125.0,4583.0,78430.0,-5.0,0.0,9.0,BUY
4,20230628.0,90001.0,476.0,13125.0,13125.0,13130.0,13125.0,4592.0,78430.0,0.0,5.0,9.0,BUY
5,20230628.0,90001.0,960.0,13125.0,13120.0,13125.0,13125.0,4601.0,78435.0,-10.0,3.0,7.0,BUY


In [7]:
df_buy = df_pre[df_pre['direction'] == 'BUY']
df_sell = df_pre[df_pre['direction'] == 'SELL']

print(f"Buy total {abs(df_buy.dVOL.sum())}, sell total {abs(df_sell.dVOL.sum())}.")

Buy total 16891.0, sell total 12354.0.
